##### 对所有静态分析结果进行统计分析

In [ ]:
import os
import json
import pprint

static_res_dir = '/root/minidroid/result/sensi_apis_trigger_path_11w'
pp_dir = '/root/minidroid/dataset/privacy_policy_11w'
all_res = {}
for file in os.listdir(static_res_dir):
    with open(os.path.join(static_res_dir, file), 'r') as fp:
        data = json.load(fp)
    all_res[file.replace('.json', '')] = data

miniset = set()
sensi_apis_count = {}
reachable_sensi_apis_count = {}
unreachable_sensi_apis_count = {}

for miniapp in all_res.keys():
    res = all_res[miniapp]
    for page in res.keys():
        apis = res[page]
        for api in apis.keys():
            if api not in sensi_apis_count.keys():
                sensi_apis_count[api] = 1
            else:
                sensi_apis_count[api] += 1
            if apis[api] is None:
                if api not in unreachable_sensi_apis_count.keys():
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] = 1
                else:
                    miniset.add(miniapp)
                    unreachable_sensi_apis_count[api] += 1
            else:
                if api not in reachable_sensi_apis_count.keys():
                    reachable_sensi_apis_count[api] = 1
                else:
                    reachable_sensi_apis_count[api] += 1

# pprint.pprint(sensi_apis_count)
# pprint.pprint(reachable_sensi_apis_count)
pprint.pprint(unreachable_sensi_apis_count)
# print(len(miniset))

# counts = 0
# for count in unreachable_sensi_apis_count.values():
#     counts += count
# print(counts)

##### 对所有静态分析结果进行一致性分析

In [ ]:
import config as config

class ConsistencyAnalyzer():
    def __init__(self, appid, sensi_apis, pp_guidelines):
        self.appid = appid
        self.sensi_apis = sensi_apis
        self.pp_guidelines = pp_guidelines
        self.reachable_sensi_apis = set()
        self.unrechable_sensi_apis = set()
        self.reachable_scopes = set()
        self.pp_scopes = self.privacy_policy_to_scopes()
        self.set_reachable_sensi_apis()
        self.res = {
            'appid': self.appid,
            'app_nickname': self.pp_guidelines['app_nickname'],
            'reachable_sensi_apis': list(self.reachable_sensi_apis),
            'unreachable_sensi_apis': list(self.unrechable_sensi_apis),
            'pp_scopes': list(self.pp_scopes),
            'redundant_scopes': [],
            'missing_scopes': []
        }
        self.consistency_analysis()
    
    def set_reachable_sensi_apis(self):
        for page in self.sensi_apis.keys():
            for sensi_api in self.sensi_apis[page].keys():
                if self.sensi_apis[page][sensi_api] == None:
                    self.unrechable_sensi_apis.add(sensi_api)
                else:
                    self.reachable_sensi_apis.add(sensi_api)
                    self.reachable_scopes.add(config.SENSITIVE_API[sensi_api])

    def privacy_policy_to_scopes(self):
        pp_scopes = set()
        for desc in self.pp_guidelines['privacy_detail_list']['item']:
            if '开发者将在获取你的明示同意后' in desc:
                scope = desc.split('，')[-1]
                pp_scopes.add(scope)
            elif '用于' in desc:
                scope = desc.split('，')[0].replace('开发者', '')
                pp_scopes.add(scope)
        return pp_scopes
    
    def consistency_analysis(self):
        redundant_scopes = list(self.pp_scopes - self.reachable_scopes)
        missing_scopes = list(self.reachable_scopes - self.pp_scopes)
        if len(redundant_scopes):
            self.res['redundant_scopes'] = redundant_scopes
        else:
            self.res['redundant_scopes'] = None
        if len(missing_scopes):
            self.res['missing_scopes'] = missing_scopes
        else:
            self.res['missing_scopes'] = None


consistency_res = []
for file in os.listdir(static_res_dir):
    try:
        appid = file.replace('.json', '')
        with open(os.path.join(static_res_dir, file), 'r') as fp:
            sensi_apis = json.load(fp=fp)
        with open(os.path.join(pp_dir, file.replace('-pc', '')), 'r') as fp:
            pp_guidelines = json.load(fp=fp)
        analyzer = ConsistencyAnalyzer(appid=appid, sensi_apis=sensi_apis, pp_guidelines=pp_guidelines)
        res = analyzer.res
        if res['redundant_scopes'] != None:
            consistency_res.append(res)
    except Exception as e:
        pass

with open('/root/minidroid/result/redundant_scopes_res_11w.json', 'w') as fp:
    json.dump(consistency_res, fp=fp, indent=4, ensure_ascii=False)
    

##### 对一致性分析结果进行统计分析

In [18]:
with open('/root/minidroid/result/redundant_scopes_res_11w.json', 'r') as fp:
    data = json.load(fp=fp)

redundant_scopes = {}
missing_scopes = {}

redundant_scope_miniapps = {}
missing_scope_miniapps = {}

miniapp_redundant_count = {}
miniapp_missing_count = {}


for miniapp in data:
    if miniapp['redundant_scopes'] != None:
        if len(miniapp['redundant_scopes']) not in miniapp_redundant_count.keys():
            miniapp_redundant_count[len(miniapp['redundant_scopes'])] = set()
            miniapp_redundant_count[len(miniapp['redundant_scopes'])].add(miniapp['appid'])
        else:
            miniapp_redundant_count[len(miniapp['redundant_scopes'])].add(miniapp['appid'])
        for scope in miniapp['redundant_scopes']:
            if scope not in redundant_scopes.keys():
                redundant_scopes[scope] = 1
                redundant_scope_miniapps[scope] = [miniapp['appid']]
            else:
                redundant_scopes[scope] += 1
                redundant_scope_miniapps[scope].append(miniapp['appid'])
    if miniapp['missing_scopes'] != None:
        if len(miniapp['missing_scopes']) not in miniapp_missing_count.keys():
            miniapp_missing_count[len(miniapp['missing_scopes'])] = set()
            miniapp_missing_count[len(miniapp['missing_scopes'])].add(miniapp['appid'])
        else:
            miniapp_redundant_count[len(miniapp['missing_scopes'])].add(miniapp['appid'])
        for scope in miniapp['missing_scopes']:
            if scope not in missing_scopes.keys():
                missing_scopes[scope] = 1
                missing_scope_miniapps[scope] = [miniapp['appid']]
            else:
                missing_scopes[scope] += 1
                missing_scope_miniapps[scope].append(miniapp['appid'])

# pprint.pprint(redundant_scopes)
# pprint.pprint(redundant_scope_miniapps)

pprint.pprint(missing_scopes)
pprint.pprint(missing_scope_miniapps)

# pprint.pprint(miniapp_redundant_count)
# pprint.pprint(miniapp_missing_count)

# missing_counts = 0
# for count in missing_scopes.values():
#     missing_counts += count
# print(missing_counts)


{'使用你的相册（仅写入）权限': 140,
 '使用你的通讯录（仅写入）权限': 23,
 '收集你的位置信息': 48,
 '收集你的发票信息': 6,
 '收集你的地址': 74,
 '收集你的微信昵称、头像': 76,
 '收集你的微信运动步数': 6,
 '收集你选中的文件': 5,
 '收集你选中的照片或视频信息': 146,
 '访问你的摄像头': 7,
 '访问你的蓝牙': 11,
 '访问你的麦克风': 1}
{'使用你的相册（仅写入）权限': ['wx6459f2843310f4ea-pc',
                   'wx6f3ff472040a4593-pc',
                   'wx5c2bea2b55bdb150-pc',
                   'wx5a406a62470e379c-pc',
                   'wx58d51acca33ceccb-pc',
                   'wx60fdb6b29adaaa7d-pc',
                   'wx5bdc567cd19080b0-pc',
                   'wx1c6a576df3f815d1-pc',
                   'wx56bf8c5c3145f064-pc',
                   'wx5cb99d4fc2314cb5-pc',
                   'wx592588d8c8e6a618-pc',
                   'wx6c6c8d41d30af836-pc',
                   'wx5c36543ca8923dca-pc',
                   'wx61b2525b669387ef-pc',
                   'wxdc4fadbce4008f5a-pc',
                   'wx5d939a8f8475646b-pc',
                   'wx6c02c06634b0032f-pc',
                   'wx5a57d0cb745e4e